In [1]:
# Pre-Processing

In [16]:
import pandas as pd
import os

In [17]:
# data preview
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat.head(10)

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,1.0,Fernando Abad*\abadfe01,28.0,OAK,AL,7,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
1,2.0,Bobby Abreu*\abreubo01,40.0,NYM,NL,78,155,133,12,33,...,0.338,0.680,97.0,45,3,0,0,2,0,H9/7D
2,3.0,José Abreu\abreujo02,27.0,CHW,AL,145,622,556,80,176,...,0.581,0.964,173.0,323,14,11,0,4,15,*3D/H
3,4.0,Tony Abreu#\abreuto01,29.0,SFG,NL,3,4,4,0,0,...,0.000,0.000,-100.0,0,1,0,0,0,0,/45H
4,5.0,Alfredo Aceves\aceveal01,31.0,NYY,AL,4,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5,6.0,Dustin Ackley*\ackledu01,26.0,SEA,AL,143,542,502,64,123,...,0.398,0.692,99.0,200,10,3,3,2,1,*7H/D
6,7.0,Cristhian Adames#\adamecr01,22.0,COL,NL,7,15,15,1,1,...,0.067,0.133,-65.0,1,1,0,0,0,0,/64H
7,8.0,Lane Adams\adamsla01,24.0,KCR,AL,6,3,3,1,0,...,0.000,0.000,-100.0,0,0,0,0,0,0,/H8D7
8,9.0,Matt Adams*\adamsma01,25.0,STL,NL,142,563,527,55,152,...,0.457,0.779,116.0,241,9,3,0,7,5,*3H
9,10.0,Mike Adams\adamsmi03,35.0,PHI,NL,22,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1


In [18]:
# removes asterisks and baseball ref id's, i.e. keeping firstname + lastname
def easy_name(name):
    name = name.split("\\")[0]
    if name[-1] == "#" or name[-1] == "*":
        name = name[:-1]
    return name

# loads, transforms, and cleans given data
market_data = pd.read_csv(os.path.join("data", "mlb_value_ranking.csv"))
def data_etl(dat, year):
    # Remove last row: league averages
    dat = dat.iloc[:-1]
    
    # Add team market sizes
    dat = dat.merge(market_data, on="Tm", how = "left")
    
    # Easy name
    dat["Name"] = dat["Name"].apply(easy_name)
    
    # Add Year Column, ID Column
    dat["Year"] = year
    dat["ID"] = dat["Name"] + dat["Year"].apply(str)
    
    # Remove dupes for multi-team players, calculate average market size
    for index, row in dat.iterrows():
        if row["Tm"] == "TOT":
            player_dat = dat[(dat["ID"] == row["ID"])]
            dat.loc[index, "Value Ranking"] = sum(player_dat["Value Ranking"])/(len(player_dat) - 1)
            dat = dat.drop(dat[(dat['Name'] == row["Name"]) & (dat["Tm"] != "TOT")].index) 
    
    # Exclude players with < 50 plate appearances
    dat = dat[dat["PA"] >= 50]
    
    return dat


In [19]:
# ETL and concatenate statistics from all years
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat = data_etl(dat, 2014)
print(len(dat))
for i in [2015, 2016, 2017, 2018, 2019, 2021]:
    dat_new = pd.read_csv(os.path.join("data", "mlb_data_" + str(i) + ".csv"))
    dat_new = data_etl(dat_new, i)
    dat = pd.concat([dat, dat_new])
    print(len(dat))

# Drop columns that we don't intend on using
cols_to_drop = ['Rk', 'Lg', 'PA', 'AB', '2B', '3B', 'CS', 'BB', 'OBP', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
dat.drop(cols_to_drop, axis=1, inplace=True)

# Exclude pitchers and rm position column
dat = dat[dat["Pos Summary"] != "1"]
dat = dat[dat["Pos Summary"] != "/1"]
dat.drop('Pos Summary', axis=1, inplace=True)

dat.head(10)

560
1108
1652
2200
2751
3321
3878


,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,BA,SLG,OPS,Value Ranking,Year,ID
1,Bobby Abreu,40.0,NYM,78,12,33,1,14,1,21,0.248,0.338,0.680,6.0,2014,Bobby Abreu2014
2,José Abreu,27.0,CHW,145,80,176,36,107,3,131,0.317,0.581,0.964,14.0,2014,José Abreu2014
5,Dustin Ackley,26.0,SEA,143,64,123,14,65,8,90,0.245,0.398,0.692,16.0,2014,Dustin Ackley2014
8,Matt Adams,25.0,STL,142,55,152,15,68,3,114,0.288,0.457,0.779,7.0,2014,Matt Adams2014
10,Jim Adduci,29.0,TEX,44,13,17,1,8,3,27,0.168,0.228,0.467,13.0,2014,Jim Adduci2014
11,Ehire Adrianza,24.0,SFG,53,10,23,0,5,1,22,0.237,0.299,0.578,5.0,2014,Ehire Adrianza2014
14,Nick Ahmed,24.0,ARI,25,9,14,1,4,0,10,0.200,0.271,0.504,20.0,2014,Nick Ahmed2014
16,Arismendy Alcantara,22.0,CHC,70,31,57,10,29,8,93,0.205,0.367,0.621,4.0,2014,Arismendy Alcantara2014
18,Abraham Almonte,25.0,TOT,59,19,47,3,15,4,60,0.230,0.333,0.609,16.5,2014,Abraham Almonte2014
22,Yonder Alonso,27.0,SDP,84,27,64,7,27,6,36,0.240,0.397,0.682,17.0,2014,Yonder Alonso2014


In [20]:
# Make sure all years are in data
dat[dat["Name"] == "Kolten Wong"]

,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,BA,SLG,OPS,Value Ranking,Year,ID
1379,Kolten Wong,23.0,STL,113,52,100,12,42,20,71,0.249,0.388,0.680,7.0,2014,Kolten Wong2014
1472,Kolten Wong,24.0,STL,150,71,146,11,61,15,95,0.262,0.386,0.707,7.0,2015,Kolten Wong2015
1444,Kolten Wong,25.0,STL,121,39,75,5,23,7,52,0.240,0.355,0.682,7.0,2016,Kolten Wong2016
1424,Kolten Wong,26.0,STL,108,55,101,4,42,8,60,0.285,0.412,0.788,7.0,2017,Kolten Wong2017
1510,Kolten Wong,27.0,STL,127,41,88,9,38,6,60,0.249,0.388,0.720,7.0,2018,Kolten Wong2018
1526,Kolten Wong,28.0,STL,148,61,136,11,59,24,83,0.285,0.423,0.784,7.0,2019,Kolten Wong2019
1695,Kolten Wong,30.0,MIL,116,70,121,14,50,12,83,0.272,0.447,0.783,24.0,2021,Kolten Wong2021


In [21]:
# ETL advanced data
adv_dat = pd.read_csv(os.path.join("data", "mlb_adv_data_2014.csv"))
adv_dat = data_etl(adv_dat, 2014)

for i in [2015, 2016, 2017, 2018, 2019, 2021]:
    dat_new = pd.read_csv(os.path.join("data", "mlb_adv_data_" + str(i) + ".csv"))
    dat_new = data_etl(dat_new, i)
    adv_dat = pd.concat([adv_dat, dat_new])

# Remove columns that we don't plan on using
adv_dat = adv_dat[["ID", "WPA", "RS%", "SB%", "ISO"]]

# Join with first data
dat = dat.merge(adv_dat, on="ID", how="left")
dat.head(10)

,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,BA,SLG,OPS,Value Ranking,Year,ID,WPA,RS%,SB%,ISO
0,Bobby Abreu,40.0,NYM,78,12,33,1,14,1,21,0.248,0.338,0.680,6.0,2014,Bobby Abreu2014,0.2,21%,100%,0.090
1,José Abreu,27.0,CHW,145,80,176,36,107,3,131,0.317,0.581,0.964,14.0,2014,José Abreu2014,4.4,22%,75%,0.264
2,Dustin Ackley,26.0,SEA,143,64,123,14,65,8,90,0.245,0.398,0.692,16.0,2014,Dustin Ackley2014,0.0,34%,67%,0.153
3,Matt Adams,25.0,STL,142,55,152,15,68,3,114,0.288,0.457,0.779,7.0,2014,Matt Adams2014,1.5,24%,60%,0.169
4,Jim Adduci,29.0,TEX,44,13,17,1,8,3,27,0.168,0.228,0.467,13.0,2014,Jim Adduci2014,-0.9,43%,75%,0.059
5,Ehire Adrianza,24.0,SFG,53,10,23,0,5,1,22,0.237,0.299,0.578,5.0,2014,Ehire Adrianza2014,-0.5,30%,50%,0.062
6,Nick Ahmed,24.0,ARI,25,9,14,1,4,0,10,0.200,0.271,0.504,20.0,2014,Nick Ahmed2014,-0.6,40%,0%,0.071
7,Arismendy Alcantara,22.0,CHC,70,31,57,10,29,8,93,0.205,0.367,0.621,4.0,2014,Arismendy Alcantara2014,-1.1,32%,62%,0.162
8,Abraham Almonte,25.0,TOT,59,19,47,3,15,4,60,0.230,0.333,0.609,16.5,2014,Abraham Almonte2014,-0.2,27%,57%,0.103
9,Yonder Alonso,27.0,SDP,84,27,64,7,27,6,36,0.240,0.397,0.682,17.0,2014,Yonder Alonso2014,-0.4,27%,86%,0.157


In [22]:
# Add all-star data
as_dat = pd.read_csv(os.path.join("data", "all_star_data.csv"))
ids = []
for index, row in as_dat.iterrows():
    ids.append(str(row["Name"]) + str(row["Year"]))

allstars = []
for index, row in dat.iterrows():
    if row["ID"] in ids:
        allstars.append(1)
    else:
        allstars.append(0)
dat["allstars"] = allstars
dat.head(10)

,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,...,SLG,OPS,Value Ranking,Year,ID,WPA,RS%,SB%,ISO,allstars
0,Bobby Abreu,40.0,NYM,78,12,33,1,14,1,21,...,0.338,0.680,6.0,2014,Bobby Abreu2014,0.2,21%,100%,0.090,0
1,José Abreu,27.0,CHW,145,80,176,36,107,3,131,...,0.581,0.964,14.0,2014,José Abreu2014,4.4,22%,75%,0.264,1
2,Dustin Ackley,26.0,SEA,143,64,123,14,65,8,90,...,0.398,0.692,16.0,2014,Dustin Ackley2014,0.0,34%,67%,0.153,0
3,Matt Adams,25.0,STL,142,55,152,15,68,3,114,...,0.457,0.779,7.0,2014,Matt Adams2014,1.5,24%,60%,0.169,0
4,Jim Adduci,29.0,TEX,44,13,17,1,8,3,27,...,0.228,0.467,13.0,2014,Jim Adduci2014,-0.9,43%,75%,0.059,0
5,Ehire Adrianza,24.0,SFG,53,10,23,0,5,1,22,...,0.299,0.578,5.0,2014,Ehire Adrianza2014,-0.5,30%,50%,0.062,0
6,Nick Ahmed,24.0,ARI,25,9,14,1,4,0,10,...,0.271,0.504,20.0,2014,Nick Ahmed2014,-0.6,40%,0%,0.071,0
7,Arismendy Alcantara,22.0,CHC,70,31,57,10,29,8,93,...,0.367,0.621,4.0,2014,Arismendy Alcantara2014,-1.1,32%,62%,0.162,0
8,Abraham Almonte,25.0,TOT,59,19,47,3,15,4,60,...,0.333,0.609,16.5,2014,Abraham Almonte2014,-0.2,27%,57%,0.103,0
9,Yonder Alonso,27.0,SDP,84,27,64,7,27,6,36,...,0.397,0.682,17.0,2014,Yonder Alonso2014,-0.4,27%,86%,0.157,0


In [23]:
# Save master data file
dat.to_csv("master_data.csv")